In [1]:
import random
import numpy as np
import tensorflow as tf
import os

# Set seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [ ]:
import pandas as pd
import numpy as np
import os

meta = pd.read_csv('Train_Test_Split.csv')
meta = meta[meta['label'].isin(['AD', 'Healthy'])]

train_meta = meta[meta['split'] == 'train']
test_meta = meta[meta['split'] == 'test']

def load_and_segment(subject_id, data_dir='Data_sampled_128HZ', segment_len=1024):
    file_path = os.path.join(data_dir, f"{subject_id}_data.npy")
    data = np.load(file_path)
    _, time_steps = data.shape
    num_segments = time_steps // segment_len
    if num_segments == 0:
        return np.empty((0, 19, segment_len))
    data = data[:, :num_segments * segment_len]
    segments = data.reshape(19, num_segments, segment_len).transpose(1, 0, 2)
    return segments

def process_data(meta_df, data_dir='Data_sampled_128HZ'):
    X = []
    y = []
    label_map = {'AD': 1, 'Healthy': 0}
    for _, row in meta_df.iterrows():
        segments = load_and_segment(row['subject_id'], data_dir)
        if segments.shape[0] == 0:
            continue
        X.append(segments)
        label = label_map[row['label']]
        one_hot = np.eye(2)[label]
        y.extend([one_hot] * segments.shape[0])
    X = np.concatenate(X, axis=0)
    y = np.array(y)
    return X, y
X_train, y_train = process_data(train_meta)
X_test, y_test = process_data(test_meta)
X_train = (X_train * 1e6) - np.mean(X_train * 1e6, axis=2, keepdims=True)
X_test = (X_test * 1e6) - np.mean(X_test * 1e6, axis=2, keepdims=True)

In [ ]:
from scipy.signal import welch

def compute_spectral_features(X, fs=128, nperseg=256):
    num_segments, num_channels, num_samples = X.shape
    psd_all = []

    for seg in X:
        seg_psd = []
        for ch in seg:
            freqs, psd = welch(ch, fs=fs, nperseg=nperseg)
            seg_psd.append(psd)
        psd_all.append(seg_psd)

    psd_all = np.array(psd_all)  
    psd_all = np.log1p(psd_all)  
    return psd_all, freqs

X_train_spec, freqs = compute_spectral_features(X_train)
X_test_spec, _ = compute_spectral_features(X_test)

X_train_spec = X_train_spec[..., np.newaxis]  # shape: (N, 19, freq_bins, 1)
X_test_spec = X_test_spec[..., np.newaxis]


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def create_spectral_cnn(input_shape, dropout_rate=0.3, dense_dim=128):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(dense_dim, activation='relu'),
        Dropout(dropout_rate),

        Dense(2, activation='softmax')
    ])
    return model

input_shape = X_train_spec.shape[1:] 
model = create_spectral_cnn(input_shape)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
]

history = model.fit(
    X_train_spec, y_train,
    validation_data=(X_test_spec, y_test),
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.6927 - loss: 1.3981 - val_accuracy: 0.5065 - val_loss: 0.6353 - learning_rate: 0.0010
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8645 - loss: 0.3131 - val_accuracy: 0.5168 - val_loss: 0.7012 - learning_rate: 0.0010
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9097 - loss: 0.2116 - val_accuracy: 0.7144 - val_loss: 0.5337 - learning_rate: 0.0010
Epoch 4/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9366 - loss: 0.1588 - val_accuracy: 0.8162 - val_loss: 0.5183 - learning_rate: 0.0010
Epoch 5/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9557 - loss: 0.1144 - val_accuracy: 0.8223 - val_loss: 0.7905 - learning_rate: 0.0010
Epoch 6/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9706 - loss: 0.0828 - val_accuracy: 0.8464 - val_loss: 0.7568 - learning_rate: 0.0010
Epoch 7/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9652 - loss: 0.

In [4]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

y_pred = model.predict(X_test_spec)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("ROC AUC:", roc_auc_score(y_true, y_pred[:, 1]))
print("Average Precision:", average_precision_score(y_true, y_pred[:, 1]))
print(classification_report(y_true, y_pred_classes))

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
ROC AUC: 0.8947969086987779
Average Precision: 0.8570660650921323
              precision    recall  f1-score   support

           0       0.78      0.83      0.81       535
           1       0.85      0.80      0.82       624

    accuracy                           0.82      1159
   macro avg       0.82      0.82      0.82      1159
weighted avg       0.82      0.82      0.82      1159



In [ ]:
output_folder = "Models"
os.makedirs(output_folder,exist_ok=True)
model_path = os.path.join(output_folder,"Final_CNNSpectral_model.keras")
model.save(model_path)